In [43]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from deap import base, creator, tools, algorithms
import random
import os
from sklearn.preprocessing import StandardScaler

In [44]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [45]:
# Load dataset
features = pd.read_csv("weatherAUS_cleaned.csv")
X = features.drop(["RainTomorrow"], axis=1).values
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = features["RainTomorrow"].values

In [46]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

In [47]:
# Define PyTorch ANN model
class ANNModel(nn.Module):
    def __init__(self, input_size, neurons_layer1, neurons_layer2, neurons_layer3, dropout_rate1, dropout_rate2):
        super(ANNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, neurons_layer1)
        self.fc2 = nn.Linear(neurons_layer1, neurons_layer2)
        self.fc3 = nn.Linear(neurons_layer2, neurons_layer3)
        self.dropout1 = nn.Dropout(dropout_rate1)
        self.fc4 = nn.Linear(neurons_layer3, 8)
        self.dropout2 = nn.Dropout(dropout_rate2)
        self.fc5 = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc4(x))
        x = self.dropout2(x)
        x = self.sigmoid(self.fc5(x))
        return x

In [48]:
# Function to train and evaluate the model
def evaluate(individual):
    neurons_layer1, neurons_layer2, neurons_layer3, dropout_rate1, dropout_rate2 = individual

    # Ensure neurons and dropout rates are within valid range
    neurons_layer1 = int(max(16, neurons_layer1))
    neurons_layer2 = int(max(16, neurons_layer2))
    neurons_layer3 = int(max(8, neurons_layer3))
    dropout_rate1 = np.clip(dropout_rate1, 0.1, 0.5)
    dropout_rate2 = np.clip(dropout_rate2, 0.1, 0.5)

    # Initialize model and move to GPU
    model = ANNModel(X_train.shape[1], neurons_layer1, neurons_layer2, neurons_layer3, dropout_rate1, dropout_rate2).to(device)
    criterion = nn.BCELoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

    # Training loop
    model.train()
    for epoch in range(200):  # Run for 10 epochs
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()

    # Evaluate model accuracy
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_tensor)
        predictions = (predictions >= 0.5).float()
        accuracy = (predictions == y_test_tensor).float().mean().item()
        
    # Save the hyperparameters and accuracy to CSV
    result = {
        'Neurons Layer 1': neurons_layer1,
        'Neurons Layer 2': neurons_layer2,
        'Neurons Layer 3': neurons_layer3,
        'Dropout Rate 1': dropout_rate1,
        'Dropout Rate 2': dropout_rate2,
        'Accuracy': accuracy
    }

    # Append results to file after each evaluation
    file_exists = os.path.isfile("hyperparameter_tuning_results.csv")  # Check if file exists
    with open("hyperparameter_tuning_results.csv", "a") as f:
        if not file_exists:  # If file doesn't exist, write the header
            header = 'Neurons Layer 1,Neurons Layer 2,Neurons Layer 3,Dropout Rate 1,Dropout Rate 2,Accuracy\n'
            f.write(header)
        f.write(f"{neurons_layer1},{neurons_layer2},{neurons_layer3},{dropout_rate1},{dropout_rate2},{accuracy}\n")

    return (accuracy,)

In [49]:
# Define Genetic Algorithm setup
if "FitnessMax" not in creator.__dict__:
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
if "Individual" not in creator.__dict__:
    creator.create("Individual", list, fitness=creator.FitnessMax)

In [50]:
toolbox = base.Toolbox()
toolbox.register("neurons_layer1", random.randint, 16, 128)
toolbox.register("neurons_layer2", random.randint, 16, 128)
toolbox.register("neurons_layer3", random.randint, 8, 64)
toolbox.register("dropout_rate1", random.uniform, 0.1, 0.5)
toolbox.register("dropout_rate2", random.uniform, 0.1, 0.5)

In [51]:
# Define individual and population
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.neurons_layer1, toolbox.neurons_layer2, toolbox.neurons_layer3,
                  toolbox.dropout_rate1, toolbox.dropout_rate2), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [52]:
def tweak(individual, sigma=5):
    """Tweak operation: Slightly modifies individual parameters to refine the search."""
    # Unpack the individual
    neurons_layer1, neurons_layer2, neurons_layer3, dropout_rate1, dropout_rate2 = individual

    # Apply small Gaussian noise to neurons while ensuring limits
    neurons_layer1 = int(max(16, neurons_layer1 + np.random.normal(0, sigma)))
    neurons_layer2 = int(max(16, neurons_layer2 + np.random.normal(0, sigma)))
    neurons_layer3 = int(max(8, neurons_layer3 + np.random.normal(0, sigma)))

    # Apply small Gaussian noise to dropout rates while keeping within range
    dropout_rate1 = np.clip(dropout_rate1 + np.random.normal(0, 0.05), 0.1, 0.5)
    dropout_rate2 = np.clip(dropout_rate2 + np.random.normal(0, 0.05), 0.1, 0.5)

    # Assign the modified values back to the individual
    individual[0] = neurons_layer1
    individual[1] = neurons_layer2
    individual[2] = neurons_layer3
    individual[3] = dropout_rate1
    individual[4] = dropout_rate2

    return individual,


In [53]:
# Register GA functions
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tweak)  # Use tweak for mutation
toolbox.register("select", tools.selTournament, tournsize=3)

In [54]:
# Run Genetic Algorithm
population = toolbox.population(n=20)
NGEN = 50  # Number of generations

In [55]:
for gen in range(NGEN):
    print(f"Generation {gen + 1}")
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
    fits = list(map(toolbox.evaluate, offspring))
    
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    
    population = toolbox.select(offspring, k=len(population))

# Best individual
best_individual = tools.selBest(population, k=1)[0]
print(f"\nBest Hyperparameters: Neurons Layer 1={best_individual[0]}, Neurons Layer 2={best_individual[1]}, Neurons Layer 3={best_individual[2]}, Dropout Rate 1={best_individual[3]:.2f}, Dropout Rate 2={best_individual[4]:.2f}")

Generation 1
Generation 2
Generation 3
Generation 4
Generation 5
Generation 6
Generation 7
Generation 8
Generation 9
Generation 10
Generation 11
Generation 12
Generation 13
Generation 14
Generation 15
Generation 16
Generation 17
Generation 18
Generation 19
Generation 20
Generation 21
Generation 22
Generation 23
Generation 24
Generation 25
Generation 26
Generation 27
Generation 28
Generation 29
Generation 30
Generation 31
Generation 32
Generation 33
Generation 34
Generation 35
Generation 36
Generation 37
Generation 38
Generation 39
Generation 40
Generation 41
Generation 42
Generation 43
Generation 44
Generation 45
Generation 46
Generation 47
Generation 48
Generation 49
Generation 50

Best Hyperparameters: Neurons Layer 1=121.67399635000224, Neurons Layer 2=79.108830953135, Neurons Layer 3=70.12469366720248, Dropout Rate 1=0.08, Dropout Rate 2=0.20
